In [ ]:
import credentials # Api key is stored in this file, 

import pvdeg
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

First get weather data for a desired location (latitude and logitude)

In [1]:
latitude = 
longitude = 

API_KEY = credentials.API_KEY



a
